## Load Dataset

In [36]:
import pandas as pd

df = pd.read_csv('cleaned_data.csv')

In [38]:
df.head(10)

,Sentence_id,New_Sentence,Type,Sentiment,Type_Encoded,New_Sentence_No_Stopwords,Sentence_Length,Sentiment_No_Stopwords
0,GERRES15609,Author and/or Review architecture/design and o...,Responsibility,0.011667,3,Author and/or Review architecture/design techn...,17,0.080000
1,PHERES15784,Should be able to develop custom dynamic shape...,Responsibility,0.250000,3,"able develop custom dynamic shape, object scri...",10,0.250000
2,GERREQ10457,Experience in working crosslly with a larger ...,Requirement,0.053333,2,Experience working crosslly larger Engineering...,10,0.053333
3,GERSKL27235,"Previous business experience, including but no...",Skill,-0.065476,4,"Previous business experience, including limite...",14,-0.119048
4,HONSSK18415,Delivering fast and right the first time.,SoftSkill,0.245238,5,Delivering fast right first time.,5,0.245238
5,INDRES9494,Track department expenses and capital budget .,Responsibility,0.000000,3,Track department expenses capital budget .,6,0.000000
6,HONRES51852,Meet performance metrics.,Responsibility,0.000000,3,Meet performance metrics.,3,0.000000
7,UAEREQ34683,YOU MUST HAVEBachelors degree in Engineering.,Requirement,0.000000,2,MUST HAVEBachelors degree Engineering.,4,0.000000
8,UAEREQ4612,After COVID-19: Ability to travel to manufactu...,Requirement,0.000000,2,COVID-19: Ability travel manufacturing site(s)...,9,0.000000
9,INDEDU42721,Six Sigma qualification to at least Green Bel...,Education,-0.250000,0,Six Sigma qualification least Green Belt level.,7,-0.250000


In [40]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Drop columns that are not needed
df = df.drop(columns=['Sentence_id', 'New_Sentence_No_Stopwords', 'Sentence_Length'])

# Text Processing: Apply TF-IDF to 'New_Sentence'
tfidf = TfidfVectorizer(stop_words='english', max_features=1000)

# Feature engineering: Scale numeric features like Sentiment and Sentiment_No_Stopwords
scaler = StandardScaler()
X_num = df[['Sentiment', 'Sentiment_No_Stopwords']]  # Assuming these are numeric columns
X_num_scaled = scaler.fit_transform(X_num)

# Combine text features with numeric features
X_text = tfidf.fit_transform(df['New_Sentence'])  # Feature from text data

# Combine all features into one matrix
from scipy.sparse import hstack
X = hstack([X_text, X_num_scaled])

# Target variable
y = df['Type_Encoded']

# Split data into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Define Logistic Regression Model
model = LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs', random_state=42)

# Cross-validation to evaluate model performance
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation and print accuracy
accuracy_scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='accuracy')
print(f"Cross-validated Accuracy: {accuracy_scores.mean():.4f}")

# Train the model on the entire training set
model.fit(X_train, y_train)

# Predict on test set
y_pred = model.predict(X_test)

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Additional: Calculate accuracy manually
print(f"Test Accuracy: {accuracy_score(y_test, y_pred):.4f}")

Cross-validated Accuracy: 0.7234
Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.71      0.75       908
           1       0.96      0.91      0.93      1815
           2       0.59      0.58      0.59      2776
           3       0.78      0.90      0.83      3051
           4       0.58      0.45      0.50      1367
           5       0.65      0.67      0.66      1883

    accuracy                           0.72     11800
   macro avg       0.72      0.70      0.71     11800
weighted avg       0.72      0.72      0.72     11800

Test Accuracy: 0.7225


In [93]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# Define the parameter grid
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],  # Regularization strength
    'max_iter': [100, 200, 300, 500],  # Max iterations to allow convergence
    'solver': ['lbfgs', 'saga'],  # Solvers that support multinomial backend
    'multi_class': ['multinomial', 'ovr']  # Multiclass strategy
}

# Initialize Logistic Regression model
log_reg = LogisticRegression(random_state=42)

# Perform Grid Search with cross-validation
grid_search = GridSearchCV(estimator=log_reg, param_grid=param_grid, cv=5, n_jobs=-1, verbose=1, scoring='accuracy')

# Fit the grid search
grid_search.fit(X_train, y_train)

# Display the best parameters found
print(f"Best Hyperparameters: {grid_search.best_params_}")

# Evaluate the best model on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Print classification report for the best model
print("Classification Report (Best Model):")
print(classification_report(y_test, y_pred))

# Additional: Accuracy of the best model
print(f"Test Accuracy (Best Model): {accuracy_score(y_test, y_pred):.4f}")

Fitting 5 folds for each of 96 candidates, totalling 480 fits


C:\Users\muham\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Best Hyperparameters: {'C': 10, 'max_iter': 100, 'multi_class': 'ovr', 'solver': 'saga'}
Classification Report (Best Model):
              precision    recall  f1-score   support

           0       0.78      0.73      0.75       908
           1       0.96      0.91      0.93      1815
           2       0.59      0.58      0.59      2776
           3       0.78      0.90      0.84      3051
           4       0.58      0.44      0.50      1367
           5       0.65      0.67      0.66      1883

    accuracy                           0.72     11800
   macro avg       0.72      0.71      0.71     11800
weighted avg       0.72      0.72      0.72     11800

Test Accuracy (Best Model): 0.7235


# Feature Importance

## Evaluasi Feature

In [68]:
# Mengakses model terbaik setelah grid search
best_model = grid_search.best_estimator_

# Mendapatkan koefisien model terbaik
coefficients = best_model.coef_[0]  # Koefisien dari model, kita ambil array pertama jika multiclass

# Jika X_train merupakan hasil dari TF-IDF dan scaling numeric features:
# Dapatkan fitur teks
text_features = tfidf.get_feature_names_out()  # Diperlukan jika Anda menggunakan TfidfVectorizer

# Dapatkan fitur numerik yang telah di-scale
numeric_features = ['Sentiment', 'Sentiment_No_Stopwords']  # Nama fitur numerik Anda

# Gabungkan nama fitur numerik dan teks
feature_names = list(text_features) + numeric_features

In [88]:
import pandas as pd
import numpy as np

# Nama fitur numerik
numeric_features = ['Sentiment', 'Sentiment_No_Stopwords']

# Koefisien dari model terbaik
coefficients = best_model.coef_[0]  # Koefisien dari model, kita ambil array pertama jika multiclass

# Dapatkan fitur teks
# Karena fitur teks berasal dari TfidfVectorizer, kita ambil nama fitur tersebut
text_features = tfidf.get_feature_names_out()  # Diperlukan jika Anda menggunakan TfidfVectorizer

# Gabungkan fitur teks dan fitur numerik
all_features = np.concatenate([text_features, numeric_features])

# Gabungkan koefisien teks dan koefisien numerik
all_coefficients = np.concatenate([coefficients[:len(text_features)], coefficients[len(text_features):]])

# Buat dataframe untuk menampilkan fitur dan koefisien
feature_importance = pd.DataFrame({
    'Feature': all_features,
    'Coefficient': all_coefficients
})

# Tambahkan kolom untuk nilai absolut koefisien agar mudah mengurutkan
feature_importance['Abs_Coefficient'] = feature_importance['Coefficient'].abs()

# Urutkan berdasarkan nilai koefisien absolut
feature_importance = feature_importance.sort_values(by='Abs_Coefficient', ascending=False)

# Tampilkan hasilnya
print(feature_importance.head(10))  # Menampilkan 20 fitur terpenting

        Feature  Coefficient  Abs_Coefficient
998       years   -11.085634        11.085634
359  experience    -5.582729         5.582729
7       ability    -5.436891         5.436891
916        tech     5.008281         5.008281
5          abet     4.898824         4.898824
556      manage    -4.721134         4.721134
16   accredited    -4.645100         4.645100
574         mba     4.537312         4.537312
677         phd     4.532340         4.532340
412    graduate     4.485125         4.485125


In [90]:
# Menampilkan fitur dan koefisien untuk Sentiment dan Sentiment_No_Stopwords
sentiment_coefficients = feature_importance[feature_importance['Feature'].isin(numeric_features)]

# Menampilkan hasilnya
print(sentiment_coefficients)

                     Feature  Coefficient  Abs_Coefficient
1001  Sentiment_No_Stopwords     0.062770         0.062770
1000               Sentiment    -0.049646         0.049646


Berdasarkan hasil feature importance yang ditampilkan, berikut adalah interpretasi dan kesimpulannya:

Kesimpulan Berdasarkan Koefisien:

1. Fitur dengan Koefisien Negatif (Dampak Negatif pada Prediksi):

a. years (-11.085634): Kata "years" memiliki koefisien negatif yang sangat besar, yang berarti semakin banyak kemunculan kata "years" dalam teks, semakin rendah kemungkinan kategori target (tipe pekerjaan) yang diprediksi. Ini menunjukkan bahwa kata "years" mungkin berhubungan dengan pengalaman yang lebih lama, namun mungkin bukan faktor yang positif atau relevan dalam konteks kategori ini.

b. experience (-5.582729): Kata "experience" juga memiliki koefisien negatif yang signifikan. Kata ini mungkin menandakan pengalaman yang lebih lama atau banyak, tetapi dalam konteks tertentu, pengalaman bisa dikaitkan dengan suatu tipe pekerjaan atau kategori yang tidak diinginkan (misalnya pekerjaan entry-level yang lebih mengutamakan keterampilan daripada pengalaman).

c. ability (-5.436891): Koefisien negatif untuk kata "ability" menunjukkan bahwa semakin sering kata ini muncul, semakin rendah peluang kategori target tertentu. Hal ini bisa menunjukkan bahwa keberadaan kata "ability" mungkin berhubungan dengan tipe pekerjaan yang tidak relevan atau tidak cocok dengan tujuan klasifikasi.

2. Fitur dengan Koefisien Positif (Dampak Positif pada Prediksi):

a. tech (5.008281): Kata "tech" menunjukkan koefisien positif, yang berarti bahwa kemunculan kata ini dapat meningkatkan kemungkinan prediksi kategori tertentu. "Tech" mungkin berhubungan dengan industri teknologi, yang menunjukkan bahwa teks dengan kata ini mungkin berkaitan dengan pekerjaan atau kategori yang relevan dengan bidang teknologi.

b. abet (4.898824): Kata ini menunjukkan koefisien positif. Meskipun konteks kata "abet" bisa lebih spesifik, secara umum ini menunjukkan bahwa kemunculan kata ini meningkatkan kemungkinan kategori tertentu, yang bisa berhubungan dengan pendidikan atau kualifikasi yang relevan.
manage (4.721134): Kata "manage" dengan koefisien positif menunjukkan bahwa kemampuan manajerial bisa sangat relevan dalam konteks prediksi kategori, misalnya untuk posisi manajerial atau level lebih tinggi dalam pekerjaan.

c. accredited (4.645100): Kata ini mungkin berhubungan dengan sertifikasi atau akreditasi tertentu, yang dapat menjadi indikator penting dalam prediksi kategori terkait pekerjaan atau pendidikan.

d. mba (4.537312) dan phd (4.532340): Koefisien positif untuk "mba" dan "phd" menunjukkan bahwa tingkat pendidikan tinggi (Master of Business Administration dan Doctor of Philosophy) sangat relevan untuk kategori tertentu, misalnya posisi yang membutuhkan kualifikasi akademik tinggi.

e. graduate (4.485125): Kata ini juga menunjukkan koefisien positif, yang bisa menunjukkan hubungan dengan lulusan universitas atau orang yang baru menyelesaikan pendidikan mereka.

## Business Insight

1. Tingkat Pendidikan yang Relevan:

Kata-kata seperti "mba", "phd", "accredited", dan "graduate" menunjukkan bahwa kualifikasi pendidikan sangat penting dalam memprediksi tipe pekerjaan atau kategori yang lebih tinggi. Oleh karena itu, perusahaan atau organisasi yang berfokus pada posisi-posisi dengan pendidikan tinggi harus memperhatikan dan menyaring pelamar yang memiliki latar belakang pendidikan yang relevan.

2. Pengalaman dan Kemampuan yang Terkait dengan Pekerjaan:

Kata "experience" dan "ability" yang memiliki koefisien negatif menunjukkan bahwa dalam konteks prediksi ini, semakin banyak pengalaman atau kemampuan yang disebutkan, semakin rendah kemungkinan untuk mencapai kategori target tertentu. Ini bisa mengindikasikan bahwa kategori target lebih relevan dengan posisi-entry level atau posisi yang mengutamakan keterampilan daripada pengalaman kerja yang luas.

3. Industri Teknologi:

Kata "tech" yang memiliki koefisien positif menunjukkan bahwa sektor teknologi atau pekerjaan yang berkaitan dengan teknologi memiliki peluang lebih besar untuk diprediksi sebagai kategori target. Perusahaan yang bergerak di bidang teknologi sebaiknya mempertimbangkan untuk menekankan penggunaan kata-kata terkait teknologi dalam deskripsi pekerjaan atau iklan lowongan mereka.

## Action Recommendation

1. Penyaringan Berdasarkan Pendidikan:

Perusahaan yang mencari kandidat dengan kualifikasi pendidikan tinggi (seperti MBA atau PhD) dapat menggunakan hasil ini untuk memfokuskan pencarian mereka pada individu dengan latar belakang akademik yang kuat. Ini dapat diterapkan dalam proses perekrutan atau analisis aplikasi kandidat.

2. Mengelola Profil Pengalaman dan Kemampuan:

Untuk posisi yang lebih berfokus pada keterampilan atau fresh graduates, kata-kata yang berhubungan dengan pengalaman yang lebih sedikit atau kemampuan teknis (misalnya, "ability") dapat diprioritaskan. Sebaliknya, jika perusahaan mencari kandidat dengan pengalaman yang lebih banyak, mereka mungkin harus lebih selektif dalam mencari kata kunci yang relevan.

3. Fokus pada Sektor Teknologi:

Jika perusahaan beroperasi di sektor teknologi, penting untuk mengaitkan deskripsi pekerjaan dengan kata-kata seperti "tech", "accredited", "manage", serta menekankan pelatihan atau sertifikasi terkait teknologi untuk menarik pelamar yang tepat.
Pertimbangkan Penyederhanaan Pengalaman:

Jika model menunjukkan bahwa pengalaman tertentu seperti "years" atau "experience" lebih berdampak negatif, hal ini bisa menjadi sinyal bahwa organisasi mungkin harus lebih fokus pada keterampilan atau kualitas daripada hanya pengalaman formal dalam perekrutan.
Dengan demikian, evaluasi fitur yang paling penting ini memberikan wawasan tentang faktor-faktor apa saja yang paling mempengaruhi prediksi model dan bagaimana hal ini dapat diterjemahkan menjadi insight bisnis yang aplikatif dalam pengambilan keputusan strategis.